following this video first to understand backprop... https://youtu.be/VMj-3S1tku0?si=eP7Ai1CO0BHh-yeS 

also note: to get this working on vscode, needed to install virtual environments T_T 

In [13]:
import math
import numpy as np 
import matplotlib.pyplot as plt

(at first the videos covers gradients, the "speed of change" at any point), then moving onto the structure of "micrograd"

In [14]:
class Value:
    # this is the constructor function in python classes 
    def __init__(self, data, _children=(), _op='', label=''): 
        self.data = data 
        self._prev = set(_children)
        self._op = _op
        self.label = label
    
    # this defines the string REPRESENTATION of the object 
    def __repr__(self): 
        return f"Value(data={self.data})"
    
    # we need to enable adding as well of these Value objects
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), '+') # creates a new value object
        return out

    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), '*') 
        return out


In [15]:
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
a + b # doesn't work until... implemented, now a.__add__(b) under the hood
a * b # doesn't work until implementing __mul__

Value(data=-6.0)

In [16]:
# now, we want to be able to SHOW THE CONNECTIVE TISSUE - GRAPHS
# what values produce other values??

# adding children... we want to now include that as part of value.
c = Value(10.0, label='c')
d = a*b + c 
d.label = 'd'
d._prev
d._op

'+'

Now we want to be able to draw a graph of the nodes!

In [17]:
from graphviz import Digraph # Digraph stands for directed graph (ok makes sense)

# using graph traversal 
def trace(root):
    # we are indeed finding all the nodes and edges in a graph 
    nodes, edges = set(), set()

    def build(v):
        # unvisited nodes only duh 
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)

    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right

    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n)) # id is a built in python function, unique (member addr)

        # we create "records" for each node
        dot.node(name = uid, label = "{data %.4f}" % (n.data, ), shape='record')
        if n._op:
            # when this node is created by an operation 
            dot.node(name = uid + n._op, label = n._op)
            # add "operation" nodes to it 
            dot.edge(uid + n._op, uid)
    
    for n1, n2 in edges: 
        # connect n1 and n2 -- the "nodes" with the name w an edge
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [18]:
draw_dot(d) # errors i think due to graphviz not properly being set up in my python evironment. oh well.

CalledProcessError: Command '[PosixPath('dot'), '-Kdot', '-Tsvg']' died with <Signals.SIGKILL: 9>. [stderr: '']